In [1]:
# imports
import numpy as np
import keras_tuner as kt
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from nnTrain import getSplitData
from selectionPlots import findAllFilesInPath

2023-02-14 17:32:15.000411: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-14 17:32:15.253646: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-14 17:32:15.263570: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/george/root/lib
2023-02-14 17:32:15.263597: I tensorflow/compiler/xla/stream_

Welcome to JupyROOT 6.26/06


/home/george/root/lib/ROOT/_facade.py:153: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  return _orig_ihook(name, *args, **kwds)


In [2]:
def model_builder(hp):
  model = Sequential()
  hp_layer_1 = hp.Int("layer_1", min_value = 12, max_value = 30, step = 2)
  hp_layer_2 = hp.Int("layer_2", min_value = 10, max_value = 18, step = 2)
  hp_layer_3 = hp.Int("layer_3", min_value = 10, max_value = 16, step = 2)
  model.add(Dense(units = hp_layer_1, input_dim = 13, activation = "relu")) # Hidden layer
  model.add(Dense(units = hp_layer_2, activation = "relu")) # Hidden layer
  model.add(Dense(units = hp_layer_3, activation = "relu")) # Hidden layer
  model.add(Dense(1, activation = "sigmoid")) # 2 output nodes for 2 classes

  model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"]) # Compile the model

  return model

In [3]:
sampleNames = findAllFilesInPath("*.root", "nTupleGroups/")
nTupleSamples = dict.fromkeys(sampleNames, 0)
nTupleSamples["nTupleGroups/signalGroup.root"] = 1

# Tuple of variables to get from each file
variables = ["tauPtSum", "zMassSum", "metPt", "deltaRll", "deltaRtt", "deltaRttll", "deltaEtall", "deltaEtatt",
             "nJets", "deltaPhill", "deltaPhitt", "deltaPhilltt", "mmc"]

cut = "2lep"
X_train, X_test, y_train, y_test = getSplitData(nTupleSamples, variables, cut, 0)

In [4]:
# Create the tuner
tuner = kt.Hyperband(model_builder,
                      objective = "val_accuracy",
                      max_epochs = 10,
                      factor = 3,
                      directory = "hyperTuning",
                      project_name = "hyperTuning" + cut,
                      overwrite = True)

stop_early = EarlyStopping(monitor = "val_loss", patience = 5)

2023-02-14 17:33:28.861343: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-14 17:33:28.861478: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/george/root/lib
2023-02-14 17:33:28.861533: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/george/root/lib
2023-02-14 17:33:28.861578: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open sha

In [5]:
# 0.25*0.8 = 0.2 to get 20% of the data for validation
tuner.search(X_train, y_train, epochs = 10, validation_split = 0.25, callbacks = [stop_early])

Trial 30 Complete [00h 00m 05s]
val_accuracy: 0.8227888941764832

Best val_accuracy So Far: 0.8321498036384583
Total elapsed time: 00h 01m 16s
INFO:tensorflow:Oracle triggered exit


In [17]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]
# print(tuner.results_summary(num_trials = 1))
# print("---")
print("The hyperparameter search is complete. The optimal number of units were\nlayer 1:", best_hps.get("layer_1"),
      "\nlayer 2:", best_hps.get("layer_2"), "\nlayer 3:", best_hps.get("layer_3"))

Results summary
Results in hyperTuning/hyperTuning2lep
Showing 1 best trials
Trial summary
Hyperparameters:
layer_1: 22
layer_2: 16
layer_3: 10
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.8321498036384583


AttributeError: 'NoneType' object has no attribute 'get'